### Imports

In [ ]:
import pandas as pd
import numpy as np
from numpy import nan as NAd
import re

# Chapter 7. Data Cleaning and Preperation

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up 80% or more of an analyst’s time. Sometimes the way that data is stored in files or databases is not in the right format for a particular task. Many researchers choose to do ad hoc processing of data from one form to another using a general-purpose programming language, like Python, Perl, R, or Java, or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form. 

If you identify a type of data manipulation that isn’t anywhere in this book or elsewhere in the pandas library, feel free to share your use case on one of the Python mailing lists or on the pandas GitHub site. Indeed, much of the design and implementation of pandas has been driven by the needs of real-world applications. In this chapter I discuss tools for missing data, duplicate data, string manipulation, and some other analytical data transformations. In the next chapter, I focus on combining and rearranging datasets in various ways.

## Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default. 

The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected:

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

In [ ]:
string_data.isnull()

In pandas, we’ve adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data. 

The built-in Python *None* value is also treated as NA in object arrays:

In [ ]:
string_data[0] = None
string_data.isnull()

### Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful. On a Series, it returns the Series with only the non-null data and index values:

In [ ]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. dropna by default drops any row containing a missing value:

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
cleaned

Passing how =' all' will only drop rows that are all NA:

In [ ]:
data.dropna(how = 'all')

In [ ]:
to drop columns:

In [ ]:
data.dropna(axis = 1, how = 'all')

Another related way to filer out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observatins. You can indicate this with the thresh argument:

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

In [ ]:
df.dropna()

In [ ]:
df.dropna(thresh = 2)

### Filling in Missing Data

Rather than filtering put missing data (and potentially discarding other data along with it), you may want to fill in the "holes" in any number of ways. For most purposes, the fillna method is the workhorse function to use. Calling fillna with a constant replaces missing values with that value:

In [ ]:
df.fillna(0)

In [ ]:
df.fillna({1: 0.5, 2 : 0})

## Data Transformation

### Removing duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [ ]:
data = pd.DataFrame({'k1' : ['one', 'two'] * 3 + ['two'],
                     'k2' : [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

### Transforming data using a function or mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [ ]:
data = pd.DataFrame({'food' : ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon',
                               'pastrami', 'honey ham', 'nova lox'],
                     'ounces' : [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

Suppose you want to add a columnd indicating the type of animal that each food came from. Let's write down a mapping of each distinct meat type to the kind of animal:

In [ ]:
meat_to_animal = {
    'bacon' : 'pig',
    'pulled pork' : 'pig',
    'pastrami' : 'cow',
    'corned beef' : 'cow',
    'honey ham' : 'pig',
    'nova lox' : 'salmon'
}

The map method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the meats are capitalized and others are not. Thus, we need to convert each value to lowercase using the str.lower Series method:

In [ ]:
lowercased = data['food'].str.lower()
lowercased

In [ ]:
data['animal'] = lowercased.map(meat_to_animal)
data

### Replacing Values

Filling in missing data with the fillna method is a special case of more general value replacement. As you've already seen, map can be used to modify a subset of values in an object, but replace provides a simpler and more flexible way to do so. Let's consider this Series:

In [ ]:
data = pd.Series([1., -999, 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data.replace({-999 : np.nan, -1000 : 0})

### Renaming Axis Indexes

Like values in Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structure. Here's a simple example:

In [ ]:
data = pd.DataFrame(
    np.arange(12).reshape((3, 4)),
    index = ['Ohio', 'Colorado', 'New York'],
    columns = ['one', 'two', 'three', 'four']
)

transform = lambda x : x[:4].upper()

data.index = data.index.map(transform)
data

### Discretization and Binning

Continious data is often discretized or otherwise seperated into "bins" for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [ ]:
ages = [20, 22, 25, 27, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let's divide these into bins of 18 to 25, 25 to 35, 36 to 60 and 61 and older.

In [ ]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

Pandas returns a Categorical object. The output you see describes the bins computed by pandas.cut. You can treat it like an array of strings indicating the bin name; internally it contains a categories array specifying the distinct category names along with a labeling for the ages data in the codes attribute

In [ ]:
cats.codes

In [ ]:
cats.categories

In [ ]:
pd.value_counts(cats)

Note that pd.value_counts(cats) are the bin counts for the result of pandas.cut 

Consisten with mathematical notation for intervals, a parenthesis means that the side is *open*, while the square bracket means it is closed (inclusive), you can change which side is closed by passing right=False

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right = False)

You can also pass your own bin names by passing a list or array to the labels option:

In [ ]:
group_names = ['Youth', 'Young Adult', 'Middle Aged', 'Senior']
pd.cut(ages, bins, labels = group_names)

If you pass an integer number of bins to cut instead of explicit bin edges, it will compute equal-length bins based on the minimum and maxoimum values in the data.

Consider the case of some uniformly distributed data chopped into fourths:

In [ ]:
data = np.random.randn(20)
pd.cut(data, 4, precision = 2)

A closely related function is qcut, which bins the data based on sample quantiles. Depending on the distribution of the data, using *cut* will not usually result in each bin having the same number of data points. Since *qcut* uses sample quantiles instead, by definition, you will obtain roughly equal-size bins:

In [ ]:
data = np.random.randn(100)
cats = pd.qcut(data, 5)
cats

In [ ]:
pd.value_counts(cats)

You can also pass your own quantiles

In [ ]:
pd.qcut(data, [0., 0.1, 0.5, 0.9, 1.])

### Detecting and filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:

In [ ]:
col = data[2]
col[np.abs(col) > 3]

To select all rows having a value exceeding 3 og -3, you can use the *any* method on boolean DataFrame:

In [ ]:
data[(np.abs(data) > 3).any(1)]

### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the numpy.random.permutation function. Calling *permutation* with the length of the acis you want to permute produces an array of integers indicating the new ordering:

In [ ]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

In [ ]:
df

In [ ]:
df.take(sampler)

To select a random subset without replacement, you can use the sample method on Series and Dataframe.

In [ ]:
df.sample(n = 3)

To generate a sample *with* replacement (to allow repeat choices), pass replace=True to sample:

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n = 10, replace = True)
draws

### Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a "dummy" or "indicator" matrix. If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s pandas has a get_dummies function for doing this.

In [ ]:
df = pd.DataFrame(
    {'key' : ['b', 'b', 'a', 'c', 'a', 'b'],
     'data1' : range(6)}
)

pd.get_dummies(df['key'])

## String manipulation

Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. Most text operations are made simple with the string object's builtin methods. For more complex pattern matching and text manipulations, regular expressions may be needed. Panadas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionaly handling the annoyance of missing data.

### String object methods

Comma seperated string can be broken into pieces.

In [ ]:
val = 'a,b, guido'
val.split(',')

split is often combined with strip to trim whitespace

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [ ]:
first, second, third = pieces

But this is not a practical generic method. A faster and more pythonic way is to pass a list or tuple to the join method on the string ':: '

In [ ]:
'::'.join(pieces)

Other methods are concerned with locating substirngs. Using Python's keyword is the best way to detect a substring, though index and find can also be used:

In [ ]:
'guido' in val

In [ ]:
val.index(',')

In [ ]:
val.find(':')

In [ ]:
val.index(':')

Relatedly, count returns the number of occurences of a particular substring:

In [ ]:
val.count(',')

replace will substitute occurences of one pattern for another. It is commonly used to delete patterns, too, by passing an empty string:

In [ ]:
val.replace(',', '::')

### Regular expressions

Regular expressions provide a flexible way to rearch or match string patterns in a text. A single expression, commonly called a regex, is a string formed according to the regular expression language. Python's built-in re module is responsible for applying regular expressions to strings.

In [ ]:
text = "foo    bar\t bax  \tqux"
re.split('\s+', text)

when calling re.split the regular expression is first compiled then it's split.

In [ ]:
regex = re.compile('\s+')
regex

In [ ]:
regex.split(text)

If instead you wanted to get a list of all patterns matching the regex, you can use the findall method:

In [ ]:
regex.findall(text)

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+.[A-Z]{2,4}'

In [ ]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

the search method returns a special match object for the first email address in the text.

In [ ]:
m = regex.search(text)
m

In [ ]:
text[m.start():m.end()]

Relatedly, sub will return a new string with occurences of the pattern replaced by a new string:

In [ ]:
print(regex.sub('REDACTED', text))

Supposed you wanted to find email adresses and simultaneously segment each address into it's three components: username, domain name and domain suffix. To do this, put parantheses around the parts of the pattern to segment:

In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0.-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)
m = regex.match('wesm@bright.net')
m.groups()

In [ ]:
regex.findall(text)

### Vectorized String Functions in pandas

Cleaning up a messy dataset for analysis often requires a lot of string munging and regularization. To complicate matters, a column containing strings will sometimes have missing data:

In [ ]:
data = {'Dave' : 'dave@google.com', 'Steve' : 'steve@gmail.com', 'Rob' : 'rob@gmail.com', 'Wez' : np.nan}
data = pd.Series(data)

In [ ]:
data

In [ ]:
data.isnull()

You can apply string and regular expression methods to each value using map() but will fail on the NA values. To cope with this, Series has array oriented methods for string operations that skip NA values. These are accessed through Series's str attribute; 

In [ ]:
data.str.contains('gmail')

In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
data.str.findall(pattern, flags=re.IGNORECASE)